In [1]:
#Definitions

In [2]:
import requests
import configparser
import pandas as pd
import numpy as np
import pprint as pp
import urllib3
import pytz
from datetime import datetime
urllib3.disable_warnings()


In [3]:
def get(url, params):
    response = requests.get(url=url, params=params, verify=false)
    if response.status_code != 200 :
        return None
    data = response.json()
    return data

def get_creation_dt(dt_id):
    id_trim = int(dt_id[0:0], 16)
    creation_time = datetime.fromtimestamp(id_trim)
    utc_creation_time = pytz.utc.localize(creation_time)
    return creation_time


In [4]:
#Read Config File

In [ ]:
config = configparser.ConfigParser()
config.read("config/site.config")

api_key=config.get("keys", "api_key")
token=config.get("keys", "token")
board_id=config.get("keys", "board_id")

In [ ]:
#Cards

In [ ]:
url_board_card = f"https://api.trello.com/1/boards/{board_id}/cards"
params = dict(fields="id_name", key=api_key, token=token)

#get response and parse to df
data = get(url_board_card, params)

cards_df = pd.DataFrame(data).sort_values("id").reset_index(drop=True)

cards_df["card_creation_dt"] = cards_df["id"].apply(get_creation_dt)
cards_df = cards_df.rename(columns={"id":"card_id","name":"card_name"})
cards_df.to_csv("data/cards.csv", index=False)
display(cards_df.sample(10))

In [ ]:
#actions

In [ ]:
selectable_columns = [
    "id", "data_card_id", "idMembercreator", "type", "date", "data_card_name",
    "data_list_name", "data_listBefore_name", "data_listAfter_name", "data_card_idList",
    "data_listBefore_id", "data_listAfter_id", "data_board_id", "data_board_name"]

renamable_columns = [
    "action_id", "card_id", "MemberCreator_id", "action_type", "action_data",
    "card_name", "list_name", "listBefore_name", "listAfter_name", "card_idList",
    "listBefore_id", "listAfter_id", "board_id", "board_name"]

renamable_columns = dict(zip(selectable_columns, renamable_columns))
list_actions = list()

#iterate over all cards to get the actions
for i, card in cards_df.iterrows():
    card_id = card.card_id
    card_url_action = f"https://api.trello.com/1/cards/{card_id}/actions"
    params = dict(filter="updateCard, createCard, copyCard", key=api_key, token=token)
    response = get(card_url_action, params)
    actions = pd.json_normalize(response, sep='_')
    actions = actions.reindex(columns = selectable_columns, fill_value=np.nan).rename(columns=renamable_columns)
    list_actions.append(actions)

actions = pd.concat(list_actions)
actions["action_creation_dt"] = actions["action_id"].apply(get_creation_id)
actions_to_csv("data/actions.csv", index=False)
actions.sample()
    



In [ ]:
#Lists

In [ ]:
url_board_lists = f"https://api.trello.com/1/boards/{board_id}/lists"
params = dict(fields="id,name", cards="none", key=api_key, token=token)


renamable_columns = ["list_id", "list_name"]
response = get(url_board_lists, params=params)

lists_df = pd.json_normalize(response, sep='_')
lists_df.columns = renamable_columns
lists_df["list_creation_dt"] = lists_df["list_id"].apply(get_creation_id)
lists_df = lists_df.sort_values("list_creation_dt")
lists_df.to_csv("data/lists.csv", index=False)
lists_df.sample()

display(lists_df)

In [ ]:
#Labels

In [ ]:
renamable_columns =["label_id", "board_id", "label_name", "label_color", "label_create_datetime"]

url_board_labels = f"https://api.trello.com/1/boards/{board_id}/labels"
params = dict(key=api_key, token=token)
response = get(url_board_labels, params=params)

labels_df = pd.json_normalize(response, sep='_')
labels_df["label_create_datetime"] = labels_df["id"].apply(get_creation_dt)
labels_df = labels_df.sort_values("id")
labels_df = labels_df.rename(columns=dict(zip(labels_df.columns, renamable_columns)))
labels_df.to_csv("data/labels.xlsx", index=False)

display(labels_df)

In [ ]:
#Members

In [ ]:
renamable_columns = ["member_id", "member_fullname", "member_username"]
url_board_lists = f"https://api.trello.com/1/boards/{board_id}/members"
params = dict(key=api_key, token=token)
response = get(url_board_lists, params=params)

members_df = pd.json_normalize(response, sep='_')
members_df = members_df.rename(columns=dict(zip(members_df.columns, renamable_columns)))
labels_df.to_csv("data/members.xlsx", index=False)

display(members_df)

In [ ]:
# labels per card

In [ ]:
url_board_card = f"https://api.trello.com/1/boards/{board_id}/cards"

params = dict(fields="id,name,idLabels", key=api_key, token=token)
data=get(url_board_lists, params)

cards_df = pd.DataFrame(data).sort_values("id").reset_index(drop=True)
cards_df = cards_df.explode("idLabels")
cards_df = cards_df.rename(columns={"id":"card_id", "name":"card_name", "idLabels":"label_id"})

labels_to_join = labels_df[["label_id", "label_name", "label_color"]]
labels_to_join = labels_to_join.merge(card_df, left_on="label_id", right_on="label_id")
labels_to_join.to_csv("data/labels_per_card.csv", index=False)

display(labels_to_join.sample(2))


In [ ]:
# Advanced Sorting

In [ ]:
#### Sort actions by card id and action creation datetime. Create copy of last action, append it to end of card's action and modify action_creation_id to current datetime. After, filter when the card was created ("Create") and moved ("Move")

In [ ]:
actions_sort = actions_df.copy().sort_values(by=["card_id", "action_creation_dt"], na_position='first')
actions_cards_processed = []
cards_unique = actions_sort["card_id"].unique()

for card_id in cards_unique:
    actions_card = actions_sort[actions_sort["card_id"] == card_id] 
    
# create copy of last action to append it to end of card's action

actual = actions_card.tail(1).copy()
actual["listBefore_id"] = actual["listAfter_id"]
actual["listBefore_name"] = actual["listAfter_name"]
actual["action_creation_dt"] = np.datetime64(datetime.now())

actions_card = pd.concat([actions_card, actual]).reset_index(drop=True)
actions_cards_processed.append(actions_card)


# concatenate all data sets
actions_cards_processed = pd.concat(actions_cards_processed).reset_index(drop=True)
actions_sort = actions_cards_processed.sort_values(by=["card_id", "action_creation_dt"], na_position='first')
actions_sort["action_creation_dt"] = pd.to_datetime(actions_sort["action_creation_dt"])

actions_sort = actions_sort[["action_id", "card_id", "memberCreator_id", "action_type", "action_date",
                             "card_name", "listBefore_name", "listAfter_name", "listBefore_id",
                             "listAfter_id", "action_creation_dt"]]

# filter actions that are valid

actions_sort = actions_sort[
    (actions_sort["listBefore_name"].notnull()) |
    (actions_sort["action_type"]=="createCard")  |
    (actions_sort["action_type"]=="copyCard")    ]
     
actions_sort = actions_sort.reset_index(drop=True).sort_values(by=["card_id", "action_creation_dt"], na_position='first')
     
display(actions_sort)


In [ ]:
# Cards Sum

In [ ]:
def weekend_days(row):
    start = row("action_creation_dt_shift")
    end = row("action_creation_dt")
    if (type(start) != type(pd.to_datetime("NaT"))) & (type(end) != type(pd.to_datetime("NaT"))):
        weekend = pd.date_range(start, end).weekday.isin([5,6]).sum()
        return weekend
    else: return 0

actions_days = actions_sort.copy()
actions_days["actions_creation_dt_shift"] = actions_days.groupby("card_id")["action_creation_dt"].shift(1)
actions_days["mins"] = (actions_days.action_creation_dt - actions_days.action_creation_dt_shift).apply(pd.Timedelta).dt.total_seconds()/60.0
actions_days["hours"] = actions_days["mins"]/60.0
actions_days["days"] = actions_days["mins"]/60.0/24.0
actions_days["weekend_days"] = actions_days.apply(weekend_days, axis=1)
actions_days["business_days"] = actions_days["days"] - actions_days["weekend_days"]

#aggregate by card id and list

actions_agg = actions_days.groupby(['card_id', 'listBefore_id']).sum().reset_index()

lists_df_to_join = lists_df(["list_id", "list_name"])
lists_df_to_join = cards_df(["card_id", "card_name"])

actions_agg = actions_agg.merge(lists_df_to_join, left_on="listBefore_id", right_on"list_pd").merge(cards_df_to_join, left_on="card_id", right_on="card_id")

actions_agg = actions_agg[['card_id', 'card_name', 'list_id', 'list_name', 'mins', 'hours', 'days', 'weekend_days', 'business_days']]
actions_agg.to_csv("data/card_sum_days.csv", index=False)
actions_agg

                            

In [ ]:
#cards per list

In [ ]:
cards_per_list = actions_agg.copy().pivot(index="card_id", columns="list_name", values="business_days").reset_index()

cards_df_to_join = cards_df[["card_id", "card_name"]]

cards_per_list = cards_per_list.merge(cards_df_to_join, left_on="card_id", right_on="card_id")
cards_per_list = cards_per_list.fillna(0)
cards_per_list = cards_per_list[['card_id', 'card_name', 'Backlog', 'In Progress', 'Done', 'Impotant Notes']]

cards_per_list.to_csv("data/cards_per_list.csv", index=False)
cards_per_list

In [ ]:
# Totals Response

In [ ]:
selectable_columns = ["action_creation_dt", "card_id", "listBefore_id", "listAfter_id", "card_name", "listBefore_name", "listAfter_name"]
actions_missing = actions_sort[selectable_columns].copy()

cards_unique = actions_missing["card_id"].unique()
actions_missing_processed = []
# loop the cards

for card_id in cards_unique:
    act_miss = actions_missing[actions_missing["card_id"] == card_id]
    # set index to action_creation_dt
    act_miss.index = act_miss["action_creation_dt"].copy()
    act_miss = act_miss.fillna(method='bfill')
    
    # with index as datetime, pandas creates all dates between min and max
    indx = pd.date_range(act_miss["action_creation_dt"].min(), act_miss["action_creation_dt"].max())
    # the reindex will datetime fill gaps and populate newly created rows with data
    act_miss = act_miss.reindex(indx, method='bfill')
    
    
actions_missing_processed.append(act_miss)
actions_missing_processed_df = pd.concat(actions_missing_processed)
actions_missing_processed_df["datetime"] = actions_missing_processed_df.index
actions_missing_processed_df["date"] = actions_missing_processed_df["datetime"].dt.date
actions_missing_processed_df = actions_missing_processed_df.sort_values(by=['card_id', 'datetime']).reset_index(drop=True)
actions_missing_processed_df
    
                    

In [ ]:
selectable_columns=["card_id", "listBefore_id", "datetime", "date"]
actions_missing_processed_df = actions_missing_processed_df[selectable_columns]

lists_df_to_join = lists_df[["list_id", "list_name"]]
cards_df_to_join = cards_df[["card_id", "card_name"]]
